# Cloud Detection with Deep Learning

This notebook demonstrates how to use a deep learning model (U-Net) for cloud detection in remote sensing imagery using `torch` and `segmentation_models_pytorch` in Python. Unlike traditional methods (e.g., `09_cloud_masking.ipynb`), this approach leverages a convolutional neural network to segment clouds in Sentinel-2 imagery, providing higher accuracy for complex scenes.

## Prerequisites
- Install required libraries: `torch`, `segmentation_models_pytorch`, `rasterio`, `geopandas`, `numpy`, `matplotlib`, `scikit-learn` (listed in `requirements.txt`).
- A preprocessed multi-band Sentinel-2 GeoTIFF (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- A labeled dataset (e.g., `cloud_labels.shp`) with binary cloud/no-cloud annotations or a pre-generated cloud mask raster.
- Replace file paths with your own data.
- GPU recommended for faster training.

## Learning Objectives
- Load and preprocess Sentinel-2 imagery and cloud mask labels.
- Train a U-Net model for cloud segmentation.
- Evaluate model performance using Intersection over Union (IoU) and accuracy.
- Generate and visualize cloud masks for new imagery.

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import os
from rasterio.mask import mask

## Step 1: Load Imagery and Cloud Labels

Load a Sentinel-2 GeoTIFF and corresponding cloud mask labels (raster or vector).

In [ ]:
# Define file paths
raster_path = 'remote_sensing_data/sentinel_rgb.tif'  # Replace with your Sentinel-2 GeoTIFF
label_path = 'cloud_labels.tif'                      # Replace with your cloud mask raster
aoi_path = 'aoi.geojson'                             # Replace with your AOI file

# Load raster
with rasterio.open(raster_path) as src:
    raster_data = src.read(masked=True)  # Shape: (bands, height, width)
    raster_profile = src.profile
    raster_crs = src.crs

# Load cloud mask (assumes binary: 1=cloud, 0=no-cloud)
with rasterio.open(label_path) as src:
    cloud_mask = src.read(1, masked=True)  # Shape: (height, width)
    label_profile = src.profile

# Load AOI and align CRS
aoi_gdf = gpd.read_file(aoi_path)
if aoi_gdf.crs != raster_crs:
    aoi_gdf = aoi_gdf.to_crs(raster_crs)

# Crop raster and labels to AOI
with rasterio.open(raster_path) as src:
    cropped_raster, cropped_transform = mask(src, aoi_gdf.geometry, crop=True, nodata=np.nan)
with rasterio.open(label_path) as src:
    cropped_labels, _ = mask(src, aoi_gdf.geometry, crop=True, nodata=np.nan)

# Update profiles
cropped_profile = raster_profile.copy()
cropped_profile.update({
    'height': cropped_raster.shape[1],
    'width': cropped_raster.shape[2],
    'transform': cropped_transform,
    'nodata': np.nan
})

# Print basic information
print(f'Cropped raster shape: {cropped_raster.shape}')
print(f'Cropped labels shape: {cropped_labels.shape}')
print(f'Raster CRS: {raster_crs}')

## Step 2: Create Custom Dataset

Extract image patches and corresponding cloud mask patches for training.

In [ ]:
# Define custom dataset class
class CloudDataset(Dataset):
    def __init__(self, raster_data, cloud_mask, patch_size=256):
        self.raster_data = raster_data
        self.cloud_mask = cloud_mask
        self.patch_size = patch_size
        self.patches = []
        self.labels = []

        # Extract patches
        height, width = raster_data.shape[1], raster_data.shape[2]
        for i in range(0, height - patch_size + 1, patch_size//2):
            for j in range(0, width - patch_size + 1, patch_size//2):
                patch = raster_data[:, i:i+patch_size, j:j+patch_size]
                label = cloud_mask[i:i+patch_size, j:j+patch_size]
                if not np.any(np.isnan(patch)) and not np.any(np.isnan(label)):
                    self.patches.append(patch)
                    self.labels.append(label)

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        patch = self.patches[idx].astype(np.float32)
        label = self.labels[idx].astype(np.int64)
        patch = torch.from_numpy(patch)
        label = torch.from_numpy(label)
        return patch, label

# Create dataset
dataset = CloudDataset(cropped_raster, cropped_labels, patch_size=256)

# Split dataset into train and validation sets
train_idx, val_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')

## Step 3: Initialize and Train U-Net Model

Set up a U-Net model with a pre-trained backbone and train it for cloud segmentation.

In [ ]:
# Initialize U-Net model
model = smp.Unet(
    encoder_name='resnet18',
    encoder_weights='imagenet',
    in_channels=cropped_raster.shape[0],  # Number of input bands
    classes=2                            # Cloud (1) and No-Cloud (0)
).to(device='cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizer
criterion = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
train_losses, val_losses = [], []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for patches, labels in train_loader:
        patches, labels = patches.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(patches)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss / len(train_loader))

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for patches, labels in val_loader:
            patches, labels = patches.to(device), labels.to(device)
            outputs = model(patches)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            val_loss += loss.item()
    val_losses.append(val_loss / len(val_loader))

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')

# Save trained model
torch.save(model.state_dict(), 'unet_cloud_detection.pth')
print('Trained model saved to: unet_cloud_detection.pth')

## Step 4: Evaluate Model Performance

Evaluate the model on the validation set using IoU and accuracy metrics.

In [ ]:
# Define IoU metric
def iou_score(preds, labels):
    preds = preds > 0.5  # Threshold logits
    intersection = (preds & labels).sum()
    union = (preds | labels).sum()
    return intersection / union if union > 0 else 0

# Evaluate model
model.eval()
all_preds, all_labels = [], []
ious = []
with torch.no_grad():
    for patches, labels in val_loader:
        patches, labels = patches.to(device), labels.to(device)
        outputs = torch.sigmoid(model(patches))
        preds = (outputs > 0.5).cpu().numpy().astype(np.uint8)
        all_preds.extend(preds.flatten())
        all_labels.extend(labels.cpu().numpy().flatten())
        ious.append(iou_score(preds, labels.cpu().numpy()))

# Compute metrics
accuracy = accuracy_score(all_labels, all_preds)
mean_iou = np.mean(ious)
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Mean IoU: {mean_iou:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap='Blues')
plt.colorbar(label='Count')
plt.xticks([0, 1], ['No-Cloud', 'Cloud'])
plt.yticks([0, 1], ['No-Cloud', 'Cloud'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## Step 5: Predict and Visualize Cloud Mask

Apply the trained U-Net model to the entire raster to generate a cloud mask.

In [ ]:
# Predict cloud mask across the entire raster
patch_size = 256
height, width = cropped_raster.shape[1], cropped_raster.shape[2]
cloud_mask_pred = np.zeros((height, width), dtype=np.uint8)

model.eval()
with torch.no_grad():
    for i in range(0, height - patch_size + 1, patch_size//2):
        for j in range(0, width - patch_size + 1, patch_size//2):
            patch = cropped_raster[:, i:i+patch_size, j:j+patch_size].astype(np.float32)
            if not np.any(np.isnan(patch)):
                patch_tensor = torch.from_numpy(patch).unsqueeze(0).to(device)
                output = torch.sigmoid(model(patch_tensor)).cpu().numpy()
                pred = (output[0, 0] > 0.5).astype(np.uint8)
                cloud_mask_pred[i:i+patch_size, j:j+patch_size] = pred

# Visualize predicted cloud mask
plt.figure(figsize=(8, 8))
plt.imshow(cloud_mask_pred, cmap='gray')
plt.title('Predicted Cloud Mask')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

# Save predicted cloud mask as GeoTIFF
pred_profile = cropped_profile.copy()
pred_profile.update({'count': 1, 'dtype': 'uint8', 'nodata': None})
pred_output_path = 'remote_sensing_data/cloud_mask_pred.tif'
with rasterio.open(pred_output_path, 'w', **pred_profile) as dst:
    dst.write(cloud_mask_pred, 1)

print(f'Predicted cloud mask saved to: {pred_output_path}')

## Step 6: Visualize with AOI Overlay

Overlay the AOI on the predicted cloud mask and original RGB for context.

In [ ]:
# Visualize RGB with predicted cloud mask and AOI
cropped_rgb = cropped_raster[:3].transpose(1, 2, 0)
cropped_rgb = cropped_rgb / np.nanpercentile(cropped_rgb, 98) if np.nanpercentile(cropped_rgb, 98) > 0 else cropped_rgb
cropped_rgb = np.clip(cropped_rgb, 0, 1)

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cropped_rgb)
ax.imshow(cloud_mask_pred, cmap='Reds', alpha=0.5)
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)
plt.title('RGB Composite with Predicted Cloud Mask and AOI')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Next Steps

- Replace `sentinel_rgb.tif` and `cloud_labels.tif` with your own GeoTIFF and cloud mask data (e.g., from `21_download_data.ipynb`).
- Update `aoi.geojson` with your area of interest file.
- Experiment with other backbones (e.g., ResNet50, EfficientNet) or loss functions (e.g., Focal Loss) to improve performance.
- Use the predicted cloud mask in preprocessing pipelines (e.g., `24_advanced_preprocessing.ipynb`) or visualization notebooks (e.g., `23_kepler_gl_demo.ipynb`).
- Explore multi-class cloud detection (e.g., cloud types) by adjusting the number of output classes.

## Notes
- Ensure the cloud mask labels are binary (0=no-cloud, 1=cloud) and align with the input raster.
- Large datasets may require smaller patch sizes or batch sizes to manage memory.
- Pre-trained backbones improve convergence; consider freezing encoder weights for faster training on small datasets.
- See `docs/installation.md` for troubleshooting library installation.